In [21]:
import pandas as pd
import numpy as np
import openml
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import KFold,GridSearchCV
from sklearn.metrics import f1_score, make_scorer,accuracy_score
import time

# Classification for Iris Dataset

In [22]:
iris = openml.datasets.get_dataset("iris")
iris_df, iris_label, categorical_indicator, attribute_names = iris.get_data(
    target=iris.default_target_attribute, dataset_format="dataframe"
)
iris_df["class"]=iris_label
iris_x=iris_df.iloc[:,:4]
iris_df

,sepallength,sepalwidth,petallength,petalwidth,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [23]:
iris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   sepallength  150 non-null    float64 
 1   sepalwidth   150 non-null    float64 
 2   petallength  150 non-null    float64 
 3   petalwidth   150 non-null    float64 
 4   class        150 non-null    category
dtypes: category(1), float64(4)
memory usage: 5.1 KB


In [24]:
le=LabelEncoder()

iris_y=le.fit_transform(iris_label)
# wine_y=le.fit_transform(wine_label)

In [25]:
scaler = MinMaxScaler()

iris_x_scaled=scaler.fit_transform(iris_x)
# wine_x_scaled=scaler.fit_transform(wine_x)

In [26]:
iris_y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [27]:
def clasification(model,parameters,x,y):
    
    classification_df=pd.DataFrame({"Model":[],
                                    "Random State":[],
                                    "Best Parameters":[],
                                    "Best Score":[],
                                    "Execution Time":[],
                                    "f1 score":[]})
    for i in range(1,11):
        start_time = time.time()
        cv_inner = KFold(n_splits=5, shuffle=True, random_state=i)
        score = ['accuracy' ,'f1_macro']
        grid_search = GridSearchCV(model, parameters, cv=cv_inner, scoring=score, refit='accuracy', n_jobs = -1)
        grid_result = grid_search.fit(x,y)
        f1_score = grid_result.cv_results_['mean_test_f1_macro']
        execution_time = time.time() - start_time
        accuracy_Score = accuracy_Score(y,)
        print("Best: %f using %s and Execution time is : %f"% (grid_result.best_score_, grid_result.best_params_,execution_time))
        classification_df=classification_df.append({"Model":model,
                                                    "Random State":i,
                                                    "Best Parameters":grid_result.best_params_,
                                                    "Best Score":grid_result.best_score_,
                                                    "Execution Time":execution_time,
                                                    "f1 score":f1_score},ignore_index=True)
       
    if model == KNC:
        classification_df.to_csv("Iris_KNC.csv")
    elif model == LDA:
        classification_df.to_csv("Iris_LDA.csv")
    elif model == GNB:
        classification_df.to_csv("Iris_GNB.csv")
    elif model == s_v_c:
        classification_df.to_csv("Iris_SVC.csv")


    return classification_df
    
    
    
#         params = grid_result.cv_results_['params']
#         f1_score = grid_result.cv_results_['mean_test_f1_macro']

SyntaxError: invalid syntax (Temp/ipykernel_12948/1321239457.py, line 17)

## K-nearest neighbour classification

In [11]:
KNC = KNeighborsClassifier()

KNC_grid = {"n_neighbors":range(1, 21),
           "weights":['uniform', 'distance'],
           "metric":['manhattan','euclidean', 'minkowski']}

In [12]:
clasification(KNC,KNC_grid,iris_x_scaled,iris_y)

Best: 0.973333 using {'metric': 'euclidean', 'n_neighbors': 9, 'weights': 'uniform'} and Execution time is : 3.954418
Best: 0.966667 using {'metric': 'euclidean', 'n_neighbors': 6, 'weights': 'uniform'} and Execution time is : 1.083102
Best: 0.966667 using {'metric': 'euclidean', 'n_neighbors': 6, 'weights': 'uniform'} and Execution time is : 1.261626
Best: 0.973333 using {'metric': 'euclidean', 'n_neighbors': 14, 'weights': 'uniform'} and Execution time is : 1.028249
Best: 0.973333 using {'metric': 'euclidean', 'n_neighbors': 14, 'weights': 'uniform'} and Execution time is : 1.027253
Best: 0.966667 using {'metric': 'euclidean', 'n_neighbors': 5, 'weights': 'uniform'} and Execution time is : 1.014287
Best: 0.966667 using {'metric': 'euclidean', 'n_neighbors': 2, 'weights': 'uniform'} and Execution time is : 1.012285
Best: 0.966667 using {'metric': 'euclidean', 'n_neighbors': 6, 'weights': 'uniform'} and Execution time is : 0.992347
Best: 0.966667 using {'metric': 'manhattan', 'n_neighb

,Model,Random State,Best Parameters,Best Score,Execution Time,f1 score
0,KNeighborsClassifier(),1.0,"{'metric': 'euclidean', 'n_neighbors': 9, 'wei...",0.973333,3.954418,"[0.9221037336254728, 0.9221037336254728, 0.928..."
1,KNeighborsClassifier(),2.0,"{'metric': 'euclidean', 'n_neighbors': 6, 'wei...",0.966667,1.083102,"[0.9401705761969593, 0.9401705761969593, 0.931..."
2,KNeighborsClassifier(),3.0,"{'metric': 'euclidean', 'n_neighbors': 6, 'wei...",0.966667,1.261626,"[0.9384541910331384, 0.9384541910331384, 0.938..."
3,KNeighborsClassifier(),4.0,"{'metric': 'euclidean', 'n_neighbors': 14, 'we...",0.973333,1.028249,"[0.9308426681677456, 0.9308426681677456, 0.942..."
4,KNeighborsClassifier(),5.0,"{'metric': 'euclidean', 'n_neighbors': 14, 'we...",0.973333,1.027253,"[0.9471420500151151, 0.9471420500151151, 0.960..."
5,KNeighborsClassifier(),6.0,"{'metric': 'euclidean', 'n_neighbors': 5, 'wei...",0.966667,1.014287,"[0.9403688419685189, 0.9403688419685189, 0.945..."
6,KNeighborsClassifier(),7.0,"{'metric': 'euclidean', 'n_neighbors': 2, 'wei...",0.966667,1.012285,"[0.9416315383514199, 0.9416315383514199, 0.955..."
7,KNeighborsClassifier(),8.0,"{'metric': 'euclidean', 'n_neighbors': 6, 'wei...",0.966667,0.992347,"[0.9409770067505967, 0.9409770067505967, 0.953..."
8,KNeighborsClassifier(),9.0,"{'metric': 'manhattan', 'n_neighbors': 18, 'we...",0.966667,1.053181,"[0.9388938790677919, 0.9388938790677919, 0.952..."
9,KNeighborsClassifier(),10.0,"{'metric': 'euclidean', 'n_neighbors': 6, 'wei...",0.960000,1.012301,"[0.9305542939583861, 0.9305542939583861, 0.941..."


## Linear discriminant analysis 

In [13]:
LDA = LinearDiscriminantAnalysis()

LDA_grid = {"n_components":range(1,3),
           "solver":['lsqr','eigen','svd']}

In [14]:
clasification(LDA,LDA_grid,iris_x_scaled,iris_y)

Best: 0.973333 using {'n_components': 1, 'solver': 'lsqr'} and Execution time is : 0.094744
Best: 0.980000 using {'n_components': 1, 'solver': 'lsqr'} and Execution time is : 0.060838
Best: 0.973333 using {'n_components': 1, 'solver': 'lsqr'} and Execution time is : 0.058844
Best: 0.980000 using {'n_components': 1, 'solver': 'lsqr'} and Execution time is : 0.064828
Best: 0.980000 using {'n_components': 1, 'solver': 'lsqr'} and Execution time is : 0.064826
Best: 0.980000 using {'n_components': 1, 'solver': 'lsqr'} and Execution time is : 0.059844
Best: 0.980000 using {'n_components': 1, 'solver': 'lsqr'} and Execution time is : 0.058845
Best: 0.980000 using {'n_components': 1, 'solver': 'lsqr'} and Execution time is : 0.060839
Best: 0.980000 using {'n_components': 1, 'solver': 'lsqr'} and Execution time is : 0.063827
Best: 0.980000 using {'n_components': 1, 'solver': 'lsqr'} and Execution time is : 0.062832


,Model,Random State,Best Parameters,Best Score,Execution Time,f1 score
0,LinearDiscriminantAnalysis(),1.0,"{'n_components': 1, 'solver': 'lsqr'}",0.973333,0.094744,"[0.9695119929800237, 0.9695119929800237, 0.969..."
1,LinearDiscriminantAnalysis(),2.0,"{'n_components': 1, 'solver': 'lsqr'}",0.980000,0.060838,"[0.9817150170091347, 0.9817150170091347, 0.981..."
2,LinearDiscriminantAnalysis(),3.0,"{'n_components': 1, 'solver': 'lsqr'}",0.973333,0.058844,"[0.9719973954494078, 0.9719973954494078, 0.971..."
3,LinearDiscriminantAnalysis(),4.0,"{'n_components': 1, 'solver': 'lsqr'}",0.980000,0.064828,"[0.9726710239651417, 0.9726710239651417, 0.972..."
4,LinearDiscriminantAnalysis(),5.0,"{'n_components': 1, 'solver': 'lsqr'}",0.980000,0.064826,"[0.9808612440191388, 0.9808612440191388, 0.980..."
5,LinearDiscriminantAnalysis(),6.0,"{'n_components': 1, 'solver': 'lsqr'}",0.980000,0.059844,"[0.9786384289795247, 0.9786384289795247, 0.978..."
6,LinearDiscriminantAnalysis(),7.0,"{'n_components': 1, 'solver': 'lsqr'}",0.980000,0.058845,"[0.9805184914545528, 0.9805184914545528, 0.980..."
7,LinearDiscriminantAnalysis(),8.0,"{'n_components': 1, 'solver': 'lsqr'}",0.980000,0.060839,"[0.9820687695368002, 0.9820687695368002, 0.982..."
8,LinearDiscriminantAnalysis(),9.0,"{'n_components': 1, 'solver': 'lsqr'}",0.980000,0.063827,"[0.9804526079206386, 0.9804526079206386, 0.980..."
9,LinearDiscriminantAnalysis(),10.0,"{'n_components': 1, 'solver': 'lsqr'}",0.980000,0.062832,"[0.9800379572118703, 0.9800379572118703, 0.980..."


## Naïve Bayes classifier

In [15]:
GNB = GaussianNB()

GNB_grid={"var_smoothing":np.logspace(0,-9, num=100)}

In [16]:
clasification(GNB,GNB_grid,iris_x_scaled,iris_y)

Best: 0.953333 using {'var_smoothing': 0.04328761281083057} and Execution time is : 0.612364
Best: 0.953333 using {'var_smoothing': 0.0657933224657568} and Execution time is : 0.612363
Best: 0.946667 using {'var_smoothing': 0.02848035868435802} and Execution time is : 0.563497
Best: 0.953333 using {'var_smoothing': 0.2848035868435802} and Execution time is : 0.595408
Best: 0.953333 using {'var_smoothing': 0.2848035868435802} and Execution time is : 0.577454
Best: 0.953333 using {'var_smoothing': 0.23101297000831597} and Execution time is : 0.597407
Best: 0.953333 using {'var_smoothing': 0.15199110829529336} and Execution time is : 0.566483
Best: 0.953333 using {'var_smoothing': 0.1873817422860384} and Execution time is : 0.573471
Best: 0.953333 using {'var_smoothing': 0.1873817422860384} and Execution time is : 0.556511
Best: 0.946667 using {'var_smoothing': 0.1873817422860384} and Execution time is : 0.624336


,Model,Random State,Best Parameters,Best Score,Execution Time,f1 score
0,GaussianNB(),1.0,{'var_smoothing': 0.04328761281083057},0.953333,0.612364,"[0.8878386136280871, 0.8878386136280871, 0.901..."
1,GaussianNB(),2.0,{'var_smoothing': 0.0657933224657568},0.953333,0.612363,"[0.9164969017910194, 0.9308645747861434, 0.930..."
2,GaussianNB(),3.0,{'var_smoothing': 0.02848035868435802},0.946667,0.563497,"[0.9246210170081486, 0.9246210170081486, 0.917..."
3,GaussianNB(),4.0,{'var_smoothing': 0.2848035868435802},0.953333,0.595408,"[0.8940059853001031, 0.8871326821326821, 0.912..."
4,GaussianNB(),5.0,{'var_smoothing': 0.2848035868435802},0.953333,0.577454,"[0.9414967357166846, 0.9414967357166846, 0.941..."
5,GaussianNB(),6.0,{'var_smoothing': 0.23101297000831597},0.953333,0.597407,"[0.9002068302068302, 0.9115728297467427, 0.917..."
6,GaussianNB(),7.0,{'var_smoothing': 0.15199110829529336},0.953333,0.566483,"[0.9429218428281558, 0.9429218428281558, 0.942..."
7,GaussianNB(),8.0,{'var_smoothing': 0.1873817422860384},0.953333,0.573471,"[0.9099783131522262, 0.9175479725949506, 0.931..."
8,GaussianNB(),9.0,{'var_smoothing': 0.1873817422860384},0.953333,0.556511,"[0.9086023773666794, 0.9086023773666794, 0.915..."
9,GaussianNB(),10.0,{'var_smoothing': 0.1873817422860384},0.946667,0.624336,"[0.904059434929, 0.904059434929, 0.91029201242..."


## Support vector machine

In [19]:
s_v_c=SVC()

SVC_grid={"C":[0.1, 1, 10, 100, 1000],
          "kernel":["linear","poly","rbf","sigmoid"],
          "gamma":[1, 0.1, 0.01, 0.001, 0.0001]}


In [20]:
clasification(s_v_c,SVC_grid,iris_x_scaled,iris_y)

Best: 0.966667 using {'C': 10, 'gamma': 1, 'kernel': 'rbf'} and Execution time is : 0.670211
Best: 0.973333 using {'C': 1000, 'gamma': 1, 'kernel': 'linear'} and Execution time is : 0.686164
Best: 0.966667 using {'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'} and Execution time is : 0.754982
Best: 0.980000 using {'C': 100, 'gamma': 0.1, 'kernel': 'sigmoid'} and Execution time is : 0.827786
Best: 0.980000 using {'C': 100, 'gamma': 1, 'kernel': 'poly'} and Execution time is : 0.784898
Best: 0.973333 using {'C': 10, 'gamma': 1, 'kernel': 'linear'} and Execution time is : 0.728054
Best: 0.966667 using {'C': 10, 'gamma': 1, 'kernel': 'linear'} and Execution time is : 0.631314
Best: 0.973333 using {'C': 100, 'gamma': 1, 'kernel': 'linear'} and Execution time is : 0.658238
Best: 0.980000 using {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'} and Execution time is : 0.718077
Best: 0.973333 using {'C': 1000, 'gamma': 1, 'kernel': 'linear'} and Execution time is : 0.714092


,Model,Random State,Best Parameters,Best Score,Execution Time,f1 score
0,SVC(),1.0,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}",0.966667,0.670211,"[0.6993275908419265, 0.8229967015978591, 0.871..."
1,SVC(),2.0,"{'C': 1000, 'gamma': 1, 'kernel': 'linear'}",0.973333,0.686164,"[0.783646305767903, 0.818489375548199, 0.91204..."
2,SVC(),3.0,"{'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}",0.966667,0.754982,"[0.8165799014753115, 0.8232676643239303, 0.925..."
3,SVC(),4.0,"{'C': 100, 'gamma': 0.1, 'kernel': 'sigmoid'}",0.980000,0.827786,"[0.803331792390616, 0.8126040685457834, 0.8996..."
4,SVC(),5.0,"{'C': 100, 'gamma': 1, 'kernel': 'poly'}",0.980000,0.784898,"[0.8450191976103942, 0.8143815705297873, 0.926..."
5,SVC(),6.0,"{'C': 10, 'gamma': 1, 'kernel': 'linear'}",0.973333,0.728054,"[0.8797588175921733, 0.8313443118727626, 0.935..."
6,SVC(),7.0,"{'C': 10, 'gamma': 1, 'kernel': 'linear'}",0.966667,0.631314,"[0.8967556572590898, 0.8242081119251669, 0.943..."
7,SVC(),8.0,"{'C': 100, 'gamma': 1, 'kernel': 'linear'}",0.973333,0.658238,"[0.9112024593763725, 0.8184909421354435, 0.938..."
8,SVC(),9.0,"{'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}",0.980000,0.718077,"[0.847208867968055, 0.8329245711313099, 0.8957..."
9,SVC(),10.0,"{'C': 1000, 'gamma': 1, 'kernel': 'linear'}",0.973333,0.714092,"[0.8127646652864045, 0.8285707481596379, 0.887..."


In [ ]:
?SVC

## Logistic regression

In [ ]:
# def clasification(model,parameters,x,y):
#     for i in range(1,10):
#         cv_inner = KFold(n_splits=5, shuffle=True, random_state=i)
#         score = ['accuracy' ,'f1_macro']

#         grid_search = GridSearchCV(model, parameters, scoring=score, refit="accuracy", error_score=0)
#         grid_result = grid_search.fit(x,y)
#         print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

#         params = grid_result.cv_results_['params']
#         f1_score = grid_result.cv_results_['mean_test_f1_macro']

#     knc_df=pd.DataFrame({"Parameters":[],
#                          "f1_score":[]})

#     for x,y in zip(params,f1_score):
#         knc_df=knc_df.append({"Parameters":x,
#                               "f1_score":y},ignore_index = True)
        
#     return knc_df
    

In [ ]:
KNC = KNeighborsClassifier(n_jobs=-1)

n_neighbors = range(1, 21)
weights = ['uniform', 'distance']
metric = ['euclidean', 'manhattan', 'minkowski']
KNC_grid = dict(n_neighbors=n_neighbors,weights=weights,metric=metric)



In [ ]:
def clasification(model,parameters,x,y):
    cv_inner = KFold(n_splits=5, shuffle=True, random_state=1)
    score = ['accuracy' ,'f1_macro']

    grid_search = GridSearchCV(model, parameters, scoring=score, refit="accuracy", error_score=0)
    grid_result = grid_search.fit(x,y)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

    params = grid_result.cv_results_['params']
    f1_score = grid_result.cv_results_['mean_test_f1_macro']

    knc_df=pd.DataFrame({"Parameters":[],
                         "f1_score":[]})

    for x,y in zip(params,f1_score):
        knc_df=knc_df.append({"Parameters":x,
                              "f1_score":y},ignore_index = True)
        
    return knc_df

In [ ]:
clasification(LDA,LDA_grid,iris_x_scaled,iris_y)

In [ ]:
model = GaussianNB()

priors=range(1,5)
var_smoothing= np.logspace(0,-9, num=100)
parameters=dict(var_smoothing=var_smoothing)

cv_inner = KFold(n_splits=5, shuffle=True, random_state=1)
score = ['accuracy' ,'f1_macro']

grid_search = GridSearchCV(model, parameters, scoring=score, refit="accuracy", error_score=0)
grid_result = grid_search.fit(iris_x_scaled,iris_y)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

params = grid_result.cv_results_['params']
f1_score = grid_result.cv_results_['mean_test_f1_macro']

knc_df=pd.DataFrame({"Parameters":[],
                         "f1_score":[]})

for x,y in zip(params,f1_score):
    knc_df=knc_df.append({"Parameters":x,
                          "f1_score":y},ignore_index = True)

knc_df


In [ ]:
wine = openml.datasets.get_dataset("wine")
wine_df, wine_label, categorical_indicator, attribute_names = wine.get_data(
    target= wine.default_target_attribute, dataset_format="dataframe"
)
wine_df["class"]=wine_label
wine_x=wine_df.iloc[:,:13]
wine_df